<a href="https://colab.research.google.com/github/CShorten/Deep-Learning-for-Econometrics/blob/main/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Erika/Deep-Learning-for-Econometrics.git
import os
os.chdir("Deep-Learning-for-Econometrics")

from Datasets import get_csv_sentences
from DataLoader import get_batch
!pip install transformers > /dev/null

Cloning into 'Deep-Learning-for-Econometrics'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 40 (delta 11), reused 2 (delta 1), pack-reused 0
Unpacking objects: 100% (40/40), done.


In [ ]:
from sklearn.model_selection import train_test_split
xs, ys = get_csv_sentences("Biotech-First-1K.csv", "Finance-First-1K.csv", "Full Description")
train_x, test_x, train_y, test_y = train_test_split(xs, ys, test_size=0.2, random_state=42)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import time
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

device = "cuda:0"
model.to(device)

batch_size = 8
total_minibatch_steps = len(xs) // batch_size

for epoch_step in range(20):
  epoch_start = time.time()
  minibatch_step = 0
  # sample batch_size number of data points each step, loop through the whole dataset like this
  while minibatch_step < total_minibatch_steps:
    # Get the data
    x_batch, y_batch = get_batch(train_x, train_y, batch_size) # randomly select batch_size instances and their labels
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)
    # Get the loss to update the model
    optimizer.zero_grad()
    outputs = model(x_batch).logits
    loss = criterion(outputs, y_batch)
    loss.backward()
    optimizer.step()
    minibatch_step += 1
  # Report training status
  print("Epoch: " +str(epoch_step) + " ran in: " + str(time.time() - epoch_start) + " seconds.")
  print("The latest loss calculation was: " + str(loss.item()))
  print("Calculating accuracy...")
  '''
  Need to figure out how to loop through batches sequentially
  '''
  '''
  with torch.no_grad():
    model.to("cpu")
    for i in range(len(train_x)):
      outputs = model(torch.from_numpy(train_x))
      _, predicted = torch.max(outputs.logits, 1)
      correct = (predicted == train_y).sum().item()
      print(correct / len(train_x) * 100)
  '''

Epoch: 0 ran in: 53.76562023162842 seconds.
The latest loss calculation was: 0.00011151026410516351
Calculating accuracy...
